In [1]:
#importar librerias importantes

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [2]:
#Lectura de Datos y presentación

datos=pd.read_excel('Base de datos 2.0.xlsx', header=0, error_bad_lines=False)
datos.head()

,Estraus,Comuna,Comercio,Educación,Habitación,Industria (1),Servicios (2),Otros (3),Sin Considerar (3),Total,...,min dist metro,Porcentaje proposito trabajo,porcentaje proposito estudio,porcentaje proposito volver a la casa,porcentaje ir de compras,porcentaje modo privado,porcentaje modo publico,porcentaje no motorizado,Área total (m2),sector
0,1,Independencia,12650,6333,78152,30140,8368,7563,40252,183458,...,3.041252,14.4932,8.9110,48.1465,8.4019,27.6373,23.7660,40.0923,4.678599e+05,norte
1,2,Independencia,39228,37662,321692,30722,19166,23144,33313,504927,...,1.609982,18.3619,8.5748,47.6031,13.9849,20.6951,30.0959,45.9323,1.134592e+06,norte
2,3,Independencia,46808,8041,213874,66033,12047,10053,35719,392575,...,2.549236,15.2141,5.9068,55.9761,8.4050,25.3639,11.8234,60.9220,9.133002e+05,norte
3,4,Independencia,32304,10895,247644,24324,14220,12453,47382,389222,...,1.620016,21.5194,8.7828,44.8588,16.4969,23.7714,23.5462,48.6555,7.513172e+05,norte
4,5,Independencia,11346,4685,59839,53861,6338,2742,30640,169451,...,3.120056,13.8301,0.0000,78.9652,4.8030,12.0075,21.1816,57.0884,4.106979e+05,norte


In [3]:
#Transformación Variables Categóricas
datoscat= datos[['sector']]
datoscat=pd.get_dummies(datoscat, columns = ['sector'], drop_first = True)
datoscat.head(10)

,sector_norte,sector_oriente,sector_poniente,sector_sur,sector_sur-oriente
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
5,1,0,0,0,0
6,1,0,0,0,0
7,1,0,0,0,0
8,1,0,0,0,0
9,1,0,0,0,0


In [4]:
#Estandarizacion Variables continuas
datoscont =datos[['Comercio','Educación','Habitación','Industria (1)','Servicios (2)','Otros (3)',
                  'Sin Considerar (3)','Min dist hosp (km)','Min dist esc',
                 'Min dist autopista','min dist cc','min dist metro','Porcentaje proposito trabajo',
                 'porcentaje proposito estudio','porcentaje ir de compras','porcentaje modo privado',
                  'porcentaje modo publico','porcentaje no motorizado','Área total (m2)']]
scaler=preprocessing.StandardScaler()
datos_est=scaler.fit_transform(datoscont)
datos_cont_est=pd.DataFrame({'Comercio':datos_est[:,0],'Educación':datos_est[:,1],'Habitación':datos_est[:,2],'Industria':datos_est[:,3],
                  'Servicios':datos_est[:,4],'Otros':datos_est[:,5],'Sin Considerar':datos_est[:,6],
                  'Dist_Hosp':datos_est[:,7],'Dist_Esc':datos_est[:,8],'Dist_Autopista':datos_est[:,9],
                  'Dist_CC':datos_est[:,10],'Dist_Metro':datos_est[:,11],'Prop_Trabajo':datos_est[:,12],
                  'Prop_Estudio':datos_est[:,13],'Prop_Compras':datos_est[:,14],'Modo_Privado':datos_est[:,15],
                  'Modo_Público':datos_est[:,16],'Modo_NoMotorizado':datos_est[:,17],'Area':datos_est[:,18]})
datos_cont_est.head()

,Comercio,Educación,Habitación,Industria,Servicios,Otros,Sin Considerar,Dist_Hosp,Dist_Esc,Dist_Autopista,Dist_CC,Dist_Metro,Prop_Trabajo,Prop_Estudio,Prop_Compras,Modo_Privado,Modo_Público,Modo_NoMotorizado,Area
0,-0.213384,-0.315154,-0.711652,0.238135,-0.186970,-0.148238,-0.083052,-0.346437,-0.332790,-0.770076,0.753861,0.120549,-0.120446,0.249972,-0.330194,-0.041228,-0.548257,0.108071,-0.174451
1,0.543424,1.721085,0.615364,0.250041,-0.024774,0.910651,-0.146974,-0.771294,-0.268814,-0.082552,0.292602,-0.393853,0.153017,0.199148,0.482824,-0.455163,0.015938,0.470327,-0.103766
2,0.759265,-0.204142,0.027878,0.972398,-0.131708,0.020983,-0.124810,-0.621059,0.067880,-0.587749,0.278715,-0.056283,-0.069488,-0.204173,-0.329742,-0.176781,-1.612720,1.400142,-0.127226
3,0.346264,-0.018646,0.211886,0.119158,-0.099067,0.184088,-0.017370,-1.000202,-0.616679,-0.330432,-0.279046,-0.390246,0.376208,0.230592,0.848631,-0.271736,-0.567848,0.639248,-0.144399
4,-0.250515,-0.422267,-0.811437,0.723395,-0.217462,-0.475875,-0.171598,-0.513320,-0.192178,-0.893480,0.437231,0.148872,-0.167318,-1.097102,-0.854279,-0.973169,-0.778608,1.162343,-0.180511


In [5]:
#Creación de Matriz de Caracteristicas y Variable Respuesta

X=pd.concat([datoscat,datos_cont_est],axis=1)
y =datos[['Comercio 2014']]

In [10]:
#Fijación de hiperparámetros

parametros= {'criterion': ['mse','friedman_mse','mae'], 'max_depth': [None,5,10,15,20,25,30], 
             'min_samples_split': [2,3,4],'min_samples_leaf': [2,3,4,5,10,15],'max_features':['auto','sqrt','log2']}

In [11]:
#Busqueda Grid Search CV

RegressionTree_GridSearch= GridSearchCV(DecisionTreeRegressor(), parametros, verbose=10000, cv=10, scoring='r2')
RegressionTree_GridSearch.fit(X,y)

Fitting 10 folds for each of 1134 candidates, totalling 11340 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2 
[CV]  criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, score=0.647, total=   0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[CV] criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2 
[CV]  criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, score=-0.206, total=   0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[CV] criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2 
[CV]  criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, score=0.423, total=   0.0s
[Parallel(n_jobs=1)]: Done   3 

[CV]  criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=3, min_samples_split=2, score=0.002, total=   0.0s
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:    0.8s remaining:    0.0s
[CV] criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=3, min_samples_split=2 
[CV]  criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=3, min_samples_split=2, score=0.700, total=   0.0s
[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:    0.8s remaining:    0.0s
[CV] criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=3, min_samples_split=2 
[CV]  criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=3, min_samples_split=2, score=0.382, total=   0.0s
[Parallel(n_jobs=1)]: Done  37 out of  37 | elapsed:    0.8s remaining:    0.0s
[CV] criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=3, min_samples_split=2 
[CV]  criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=3, min_samples_split=2, sco

[CV] criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV]  criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=2, score=0.250, total=   0.0s
[Parallel(n_jobs=1)]: Done  65 out of  65 | elapsed:    1.4s remaining:    0.0s
[CV] criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV]  criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=2, score=0.722, total=   0.0s
[Parallel(n_jobs=1)]: Done  66 out of  66 | elapsed:    1.4s remaining:    0.0s
[CV] criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV]  criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=2, score=0.401, total=   0.0s
[Parallel(n_jobs=1)]: Done  67 out of  67 | elapsed:    1.4s remaining:    0.0s
[CV] criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=2 
[CV]

[CV]  criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=5, min_samples_split=2, score=0.234, total=   0.0s
[Parallel(n_jobs=1)]: Done  98 out of  98 | elapsed:    2.0s remaining:    0.0s
[CV] criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=5, min_samples_split=2 
[CV]  criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=5, min_samples_split=2, score=0.659, total=   0.0s
[Parallel(n_jobs=1)]: Done  99 out of  99 | elapsed:    2.1s remaining:    0.0s
[CV] criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=5, min_samples_split=2 
[CV]  criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=5, min_samples_split=2, score=0.500, total=   0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.1s remaining:    0.0s
[CV] criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=5, min_samples_split=3 
[CV]  criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=5, min_samples_split=3, sco

[CV]  criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=10, min_samples_split=2, score=0.493, total=   0.0s
[Parallel(n_jobs=1)]: Done 130 out of 130 | elapsed:    2.6s remaining:    0.0s
[CV] criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=10, min_samples_split=3 
[CV]  criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=10, min_samples_split=3, score=0.471, total=   0.0s
[Parallel(n_jobs=1)]: Done 131 out of 131 | elapsed:    2.6s remaining:    0.0s
[CV] criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=10, min_samples_split=3 
[CV]  criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=10, min_samples_split=3, score=-0.691, total=   0.0s
[Parallel(n_jobs=1)]: Done 132 out of 132 | elapsed:    2.7s remaining:    0.0s
[CV] criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=10, min_samples_split=3 
[CV]  criterion=mse, max_depth=None, max_features=auto, min_samples_leaf=10, min_samples_spli

...

[CV]  criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=3, min_samples_split=3, score=0.519, total=   0.0s
[CV] criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=3, min_samples_split=4 
[CV]  criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=3, min_samples_split=4, score=0.497, total=   0.0s
[CV] criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=3, min_samples_split=4 
[CV]  criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=3, min_samples_split=4, score=0.226, total=   0.0s
[CV] criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=3, min_samples_split=4 
[CV]  criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=3, min_samples_split=4, score=0.055, total=   0.0s
[CV] criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=3, min_samples_split=4 
[CV]  criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=3, min_samples_split=4, score=0.396, total=   0.0s
[CV] criterion

[CV]  criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=5, min_samples_split=2, score=0.112, total=   0.1s
[CV] criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=5, min_samples_split=2 
[CV]  criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=5, min_samples_split=2, score=0.580, total=   0.1s
[CV] criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=5, min_samples_split=2 
[CV]  criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=5, min_samples_split=2, score=0.174, total=   0.1s
[CV] criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=5, min_samples_split=2 
[CV]  criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=5, min_samples_split=2, score=0.629, total=   0.0s
[CV] criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=5, min_samples_split=2 
[CV]  criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=5, min_samples_split=2, score=-1.677, total=   0.1s
[CV] criterio

[CV]  criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=10, min_samples_split=3, score=0.368, total=   0.0s
[CV] criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=10, min_samples_split=3 
[CV]  criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=10, min_samples_split=3, score=-3.077, total=   0.0s
[CV] criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=10, min_samples_split=3 
[CV]  criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=10, min_samples_split=3, score=0.234, total=   0.0s
[CV] criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=10, min_samples_split=3 
[CV]  criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=10, min_samples_split=3, score=0.211, total=   0.1s
[CV] criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=10, min_samples_split=3 
[CV]  criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=10, min_samples_split=3, score=0.721, total=   0.0s
[CV]

[CV]  criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=15, min_samples_split=4, score=0.092, total=   0.0s
[CV] criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=15, min_samples_split=4 
[CV]  criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=15, min_samples_split=4, score=0.732, total=   0.0s
[CV] criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=15, min_samples_split=4 
[CV]  criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=15, min_samples_split=4, score=0.259, total=   0.0s
[CV] criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=15, min_samples_split=4 
[CV]  criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=15, min_samples_split=4, score=0.319, total=   0.0s
[CV] criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=15, min_samples_split=4 
[CV]  criterion=mae, max_depth=30, max_features=log2, min_samples_leaf=15, min_samples_split=4, score=0.581, total=   0.0s
[CV] 

GridSearchCV(cv=10, error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                             max_depth=None, max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=None,
                                             splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'criterion': ['mse', 'friedman_mse', 'mae'],
                         'max_depth': [None, 5, 10, 15, 20, 25, 30],
               

In [12]:
#Mejores hiperparametros
RegressionTree_GridSearch.best_params_

{'criterion': 'mae',
 'max_depth': None,
 'max_features': 'auto',
 'min_samples_leaf': 15,
 'min_samples_split': 2}

In [13]:
#Resultados GridSearch
RegressionTree_GridSearch.cv_results_

{'mean_fit_time': array([0.01848662, 0.0253948 , 0.02005317, ..., 0.03607941, 0.03320131,
        0.03229392]),
 'std_fit_time': array([0.00728828, 0.01135242, 0.00614297, ..., 0.01475226, 0.01498639,
        0.00944643]),
 'mean_score_time': array([0.00156481, 0.00469038, 0.00312593, ..., 0.00468512, 0.00690217,
        0.00156658]),
 'std_score_time': array([0.00469444, 0.00716468, 0.00625186, ..., 0.00715663, 0.00736801,
        0.00469973]),
 'param_criterion': masked_array(data=['mse', 'mse', 'mse', ..., 'mae', 'mae', 'mae'],
              mask=[False, False, False, ..., False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[None, None, None, ..., 30, 30, 30],
              mask=[False, False, False, ..., False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_features': masked_array(data=['auto', 'auto', 'auto', ..., 'log2', 'log2', 'log2'],
              mask=[False, False, False, ..., False,

In [14]:
#Implementación Modelo

model= DecisionTreeRegressor(criterion='mae', max_depth=None, min_samples_split=2 ,min_samples_leaf=15 ,max_features='auto')

In [15]:
#Cross Validation

Scores = cross_val_score(model, X, y.values.ravel(), cv=10, scoring=('r2'))
Scores

array([0.90111132, 0.25492937, 0.60372687, 0.81076923, 0.63215295,
       0.78063695, 0.35878986, 0.32034653, 0.67565834, 0.41208717])